In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pipeline import PipelineConfig, PipelineFactory

In [14]:
# Load
df = pd.read_csv("../data/cleaned_data_after_eda.csv")
TARGET = "Percent_Bleached"
y = df[TARGET].astype(int)
X = df.drop(columns=[TARGET])

# Column lists
numerical_cols    = X.select_dtypes(include="number").columns.tolist()
categorical_cols  = ["Exposure"]  # only this one is categorical per your note
log_cols          = []            # fill if you want log1p on specific numeric cols


In [15]:
from pipeline.weights import WeightingConfig

# Split
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

# Build config
cfg = PipelineConfig(
    numeric=numerical_cols,
    categorical=categorical_cols,
    log=log_cols,
    model_name="hgbr",                 # just this change
    # weighting=WeightingConfig(                   # mild tail emphasis
    #     scheme="tail_focus", tails=0.10, power=1.2, normalize=True
    # ),
)
# Build & train pipeline
pipe = PipelineFactory().build(cfg)
pipe.fit(X_tr, y_tr)

# Evaluate (simple R^2 example)
r2 = pipe.score(X_te, y_te)
print(f"Test R^2: {r2:.3f}")


TypeError: 'HistGradientBoostingRegressor' object is not callable

In [12]:
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error


def evaluate(y_true, y_pred):
    r2  = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    q90 = np.quantile(y_true, 0.9)
    mask = y_true >= q90
    mae_top = mean_absolute_error(y_true[mask], y_pred[mask]) if mask.any() else np.nan
    return {"R2": r2, "MAE": mae, "MAE_top10%": mae_top}

yhat = pipe.predict(X_te)
print(evaluate(y_te, yhat))

{'R2': -0.23071004667245676, 'MAE': 19.051841746248293, 'MAE_top10%': 71.89189189189189}
